In [ ]:
# Ejemplo de como armar un chat con message history.
# No estoy usando langgraph.
# https://python.langchain.com/docs/how_to/chatbots_memory/

In [ ]:
%%capture outputtext

# Colab:
!pip install langchain-community
!pip install langchain-ollama
!curl -fsSL https://ollama.com/install.sh | sh
!ollama serve &>/dev/null&

# # Local:
# import os
# environment= 'local'
# os.system('ollama serve &>/dev/null&')

In [2]:
# !ollama pull llama3.1
# !ollama pull llama3.2-vision
!ollama pull qwen2.5vl:7b

In [ ]:
from langchain_ollama import ChatOllama

model = ChatOllama(
    model = "qwen2.5vl:7b",
    # model = "llama3.2-vision",
    # model = "llama3.1",
    # model = "llama3.1:8b",
    # temperature = 0.8,  # default en llama
)


In [4]:
# Ejemplo muy básico.
# Agrego la respuesta del modelo a la lista.

messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]

ai_msg = model.invoke(messages)
print(ai_msg.content)

messages.append(('ai',ai_msg.content))
messages.append(('human','Count the letters in your response.'))

ai_msg = model.invoke(messages)
print(ai_msg.content)

J'aime le programmation.
La réponse contient 19 lettres.


In [5]:
##
## Ahora con una imagen.
##

import base64

filename = 'jackson_2_7.jpg'

url = 'https://raw.githubusercontent.com/frautn/llamita/main/data/' + filename
!curl -OL {url}


with open(filename, "rb") as image_file:
    image_data = base64.b64encode(image_file.read())

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  478k  100  478k    0     0  1762k      0 --:--:-- --:--:-- --:--:-- 1765k


In [ ]:
message = [({
    "role": "user",
    "content": [
        {
            "type": "text",
            "text": "Transcribe the text in the image using LaTeX. Don't solve the problem.",
        },
        {
            "type": "image",
            "source_type": "base64",
            "data": image_data.decode('utf8'),
            "mime_type": "image/jpg",
        },
    ],
})
]

response = model.invoke(message)
output = response.text()
output = output.strip("```latex")

print(response.text())

In [ ]:
message.append(
    {
    "role": "ai",
    "content": [
        {
            "type": "text",
            "text": output,
        },
    ]
    }
)

message.append(
    {
    "role": "user",
    "content": [
        {
            "type": "text",
            "text": "Type only item a)",
        },
    ]
    }
)

response = model.invoke(message)
print(response.text())

In [ ]:
message.append(
    {
    "role": "ai",
    "content": [
        {
            "type": "text",
            "text": response.text(),
        },
    ]
    }
)

message.append(
    {
    "role": "user",
    "content": [
        {
            "type": "text",
            "text": "Translate your last response in Spanish.",
        },
    ]
    }
)

response = model.invoke(message)
print(response.text())